# Test parsing on FB data
How well can we parse FB data? We need to parse in order to extract appositives.

In [1]:
import pandas as pd
from ast import literal_eval
entity_data_tagged = pd.read_csv('../../data/facebook-maria/combined_group_data_es_tagged.tsv', sep='\t', index_col=False)
print('%d original statuses'%(entity_data_tagged.shape[0]))
# convert lists
entity_data_tagged.loc[:, 'status_message_tags'] = entity_data_tagged.loc[:, 'status_message_tags'].apply(literal_eval)
entity_data_tagged.loc[:, 'status_message_tags_ne'] = entity_data_tagged.loc[:, 'status_message_tags_ne'].apply(literal_eval)
# get rid of extra-long statuses
max_status_len = 150
entity_data_tagged = entity_data_tagged[entity_data_tagged.loc[:, 'status_message_clean'].apply(lambda x: len(x.split(' '))) < max_status_len]
print('%d filtered statuses'%(entity_data_tagged.shape[0]))
display(entity_data_tagged.head())

/hg190/istewart6/miniconda3/envs/crisis_language/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/hg190/istewart6/miniconda3/envs/crisis_language/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


23662 original statuses
23004 filtered statuses


,group_id,status_author_id,status_message,status_lang,status_id,status_published,status_message_clean,status_message_tags,status_message_tags_ne,status_message_ne_tagged_stemmed
0,351272391991842,10159623474155515,Estoy preocupada por mi tia vicky vazquez que ...,es,351272391991842_362333787552369,2017-10-19 18:25:23,Estoy preocupada por mi tia vicky vazquez que ...,"[(Estoy, O), (preocupada, O), (por, O), (mi, O...","[(asma, CAUSE_OF_DEATH), (Dorado, ORGANIZATION)]",estoy preocup por mi tia vicky vazquez que des...
2,351272391991842,1716175958400853,En apoyo al alcalde Carlos Lopez y al municipi...,es,351272391991842_361453717640376,2017-10-17 17:38:08,En apoyo al alcalde Carlos Lopez y al municipi...,"[(En, O), (apoyo, O), (a, O), (el, O), (alcald...","[(alcalde, TITLE), (Carlos Lopez, PERSON), (Do...",en apoy a el alcalde.<ne.title> carlos_lopez.<...
3,351272391991842,141039043205101,Pueblo de Puerto Rico no se monten en las gran...,es,351272391991842_358929311226150,2017-10-11 14:14:22,Pueblo de Puerto Rico no se monten en las gran...,"[(Pueblo, O), (de, O), (Puerto, COUNTRY), (Ric...","[(Puerto Rico, COUNTRY), (America, MISC), (Pue...",puebl de puerto_rico.<ne.country> no se mont e...
5,351272391991842,2060154724214607,"Guayama, Dorado, y Piñones",es,351272391991842_357797888005959,2017-10-08 14:52:49,"Guayama, Dorado, y Piñones","[(Guayama, LOCATION), (,, O), (Dorado, O), (,,...","[(Guayama, LOCATION)]","guayama.<ne.location>, dor, y piñones.<ne.person>"
6,351272391991842,2060154724214607,Videos de Dorado dos semanas despues del Hurac...,es,351272391991842_356944798091268,2017-10-06 13:12:14,Videos de Dorado dos semanas despues del Hurac...,"[(Videos, O), (de, O), (Dorado, O), (dos, NUMB...","[(dos, NUMBER)]",vide de dor dos.<ne.number> seman despu de el ...


In [2]:
loc_types = ['COUNTRY', 'CITY', 'LOCATION']
loc_idx = entity_data_tagged.loc[:, 'status_message_tags_ne'].apply(lambda x: len([y[1] in loc_types for y in x]) > 0)
loc_idx = loc_idx[loc_idx].index.tolist()
entity_data_tagged_loc = entity_data_tagged.loc[loc_idx, :]
print('\n'.join(entity_data_tagged_loc.loc[:, 'status_message_clean'].iloc[280:300]))

Dios les Bendiga..Ojo 👀 precaución amistades, familias, vecinos Precaución...Les comparto esta información:ATENCIÓN: Absolutamente nadie le puede cobrar por llenar las ayudas de FEMA. Nos están llegando confidencias de que ciudadanos se están aprovechando de la situación (falta de energía e internet) y están cobrando hasta $25 por llenar las ayudas. ESTO ES ILEGAL Y LA PERSONA QUE LO HAGA PUEDE SER PROCESADA EN EL FORO FEDERAL. Si conoce de alguien que se esté aprovechando del más necesitado para generar algún ingreso, denúncielo. Compártelo.Gracias.
Alguien sabe si ya western union o cualquier otro Tipo de envio de dinero está funcionando cerca Del pueblo de Corozal?
El Alcalde de Corozal habla y recibe ayuda de la Alcadesa de San Juan...
Corozal necesita agua por favor tratar de ir a las casas en el Barrio Palmarejo sector febus mucha jente si agua.
Si alguien de Corozal, sabe de Las familias Avilés, Marrero de Cuchillas, Díaz de Maguelles o relacionados favor de llamar a Roberto Día

In [3]:
import re
test_locs = ['higuillar', 'dorado', 'corozal', 'palmarejo', 'barranquitas', 'san germán']
for test_loc_i in test_locs:
    test_loc_i_matcher = re.compile(test_loc_i)
    entity_data_tagged_i = entity_data_tagged[entity_data_tagged.loc[:, 'status_message_clean'].apply(lambda x: test_loc_i_matcher.search(x.lower()) is not None)]
    print('\n'.join(entity_data_tagged_i.loc[:, 'status_message_clean'].values))

Estoy preocupada por mi tia vicky vazquez que desde el dia antes del huracan no se de ella pues ella es una mujer enferma tiene que usar oxijeno y ella me dijo el dia antes del huracan que su oxijeno se avia acabado y que no se sentia bien pero que se estava dando terapias con la maquina del asma ella vive en higuillar en Dorado creo qye se llama arenar en la calle que vive al frente de su casa antes hera una vaqueria yo espero que ella este bien por favor si alguien del grupo es de esa aria y la conose digale que su sobrina la esta buscando gracias por favor de compartir
Buenas noches.. busco información del barrio higuillar en el area frente a granas lindas.. al lado del colmadito del difunto Pereda.. tengo una tía q vive ahí se llama carmín nieves.. agradeceré cualquier información de ella o de esa área en la q ella vive... gracias
Deseo saber sobre el bienestar de mi hermano Armando Sánchez y su esposa Adela Le Ravén, de mi sobrino Ián Yarel Sánchez y su esposa Nancy. Residen en Ba

Feliz Hoy Viernes 13 de Octubre 2017 me llamo mi Hija que estuvo la Cruz Roja llevando provisiones el la cancha de las Parcelas Julián Marrero de Palmarejo.
Una vista desde Palmarejo, lindo dia a todos los corozaleños del mundo 🌎
Gracias de aquí La Florida donde yo vivo al alcalde de Corozal, mi mamá vive en Palmarejo sector febus tiene 87 años y esta sola por el momemto ya a recibido 3 veces ayuda como Agua, comida y lateria se lo agradezco mucho de todo Corazón. Mi mamá Maria L Negron se lo agradecemos mucho
Sector los llanos en Palmarejo no visto ayuda alguna mi hermana tiene MA's de doce personas a su cargo incluyendo Menores
Saludos mi gente bella de Corozal. Alguien conoce mi familia? Son los Morales de Palmarejo sector las Cantera? Cualquier informacion seria agradecida 100 veces. El q los conosca por favor les dejen saber q MIKEY les envia mil besos y abrasos y Orando q todos c encuentren bien de salud. Grax y q Dios me los cuide y me los guarde a todos en Corozal y todo P.R. A

Copy and Paste! Hospitales en Puerto Rico:Funcionando salas de emergencia:•Hospital Pavia Santurce•Hospital Metropolitano en Río Piedras•Hospital San Francisco en Río Piedras•Hospital Pavia Arecibo•Hospital Pavia Yauco•Hospital Metropolitano San Germán•Hospital Perea, Mayagüez•Hima San Pablo, Bayamón Mantenga en mente que la luz en algunos de estos hospitales es intermitente y hay casos extremos que corren con prioridad. Sea paciente y gentil con el equipo que está haciendo lo posible por dar servicios.
#SANGERMÁN El alcalde de San Germán ha establecido un Stop and Go para familias del pueblo. Todos los días pueden ir al drive-thru y les dan agua y comida. Están ubicados en el parque de agua de San Germán. Díganle a sus familiares. Compartan también estos teléfonos que ya trabajan:*Manejo de Emergencias 787-892-5625 787-892-5685*Policía SG 787-892-3505
Por favor!! Si alguien sabe dónde se puede conseguir gas en San Germán o áreas cercanas se los voy a agradecer. Necesito urgente!!!
URL

Possible descriptors:

- PP => `en Dorado`, `en el area frente a granas lindas`, `La Guarida del Penon, en Rosario Peñon, en San Germán`, `Barrio negro en Corozal`, `Barrio Palmarejo de Coamo`, `sector Palmarejo de Coamo`, `Los LLANOS de Coamo Barrio Cuyon Palmarejo`, `del Cerro en Palmarejo`, `del supermercado Alcoba en Palmarejo`, `El Rancho en Palmarejo`, `en Canales de Hato Rey`, `Jardines de Barranquitas`, `Las Bromelias en Barranquitas`
- NP => `del barrio higuillar`, `Corozal PR`, `Barrio Palmarejo`, `el rio de Palmarejo, Corozal`, `sector Pancho Febus`, `Sector los llanos en Palmarejo`, `Coamo Sector Palmarejo`, `Barrio Palmarejo y Barrio Cuyon`, `Dorado Calle Norte`, `Palmarejo sector las Cantera`, `el Barrio Palmarejo sector febus`, `Los Guardias, El Rancho Palmarejo Corozal`, `Palmarejo sector los guardias`, `sector la loma`, `La Loma Sector el Tiguere Barranquitas`, `en Palo Hincado, sector Puente Roto`

How does it look when we parse these sentences?

Let's get 10 example sentences, run them through the parsers and see which one does the least badly.

- Install/run Google parser: https://cloud.google.com/python/
- Run Stanford parser:
```
cd /hg190/corpora/StanfordCoreNLP/stanford-corenlp-full-2018-02-27
java -Xmx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -serverProperties StanfordCoreNLP-spanish.properties -preload tokenize,ssplit,pos,ner,parse -status_port 9003  -port 9003 -timeout 15000
```

In [73]:
import re
import spacy
from spacy.language import DependencyParser
spacy_nlp = spacy.load('es')
from nltk.parse.corenlp import CoreNLPParser, CoreNLPDependencyParser
stanford_parser = CoreNLPDependencyParser(url='http://localhost:9003')
# stanford_parser = CoreNLPParser(url='http://localhost:9003', tagtype='ner') # Chomsky graphs
from google.cloud import language
google_cred_file = '../../data/Google_cloud/TestProject_creds.json'
google_client = language.LanguageServiceClient.from_service_account_json(google_cred_file)

def build_parse(parse_result, parse_type='spacy'):
    parse_tuples = [] # token, POS, head_index, dependency label
    # get tokens
    if(parse_type == 'spacy'):
        parse_tokens = parse_result
    elif(parse_type == 'stanford'):
        parse_tokens = sorted(parse_result.nodes.items(), key=lambda x: x[0])[1:] # skip the first token
    elif(parse_type == 'google'):
        parse_tokens = parse_result.__getattribute__('tokens')
    # get tuple for each token
    google_pos_matcher = re.compile('(?<=tag: ).+')
    google_proper_matcher = re.compile('(?<=proper: ).+')
    google_dep_label_matcher = re.compile('(?<=label: ).+')
    for token in parse_tokens:
        if(parse_type == 'spacy'):
            parse_tuple = [
                token.text, 
                token.pos_,
                token.head.i,
                token.dep_,
            ]
        elif(parse_type == 'stanford'):
            idx, tok = token
            parse_tuple = [
                tok['word'],
                tok['tag'],
                tok['head'],
                tok['rel'],
            ]
        elif(parse_type == 'google'):
#             print(token)
#             print(token.__getattribute__('part_of_speech'))
#             print(token.__getattribute__('dependency_edge'))
            pos_tag = google_pos_matcher.search(str(token.__getattribute__('part_of_speech'))).group(0)
            proper_tag = google_proper_matcher.search(str(token.__getattribute__('part_of_speech')))
            if(proper_tag is not None and proper_tag.group(0) == 'PROPER'):
                pos_tag = 'PROPN'
            parse_tuple = [token.__getattribute__('text').__getattribute__('content'), 
                           pos_tag,
                           token.__getattribute__('dependency_edge').__getattribute__('head_token_index'),
                           google_dep_label_matcher.search(str(token.__getattribute__('dependency_edge'))).group(0).lower(),
                          ]
        parse_tuples.append(parse_tuple)
    return parse_tuples

test_sents = [
    "Ella vive en Palo Hincado, sector Puente Roto.", 
    "Hola, mi esposo Efrain J Irizarry y su hermano Nelson Irizarry andan buscando a su Papa y su esposa, Mayra Rodriguez dueños de La Guarida del Penon, en Rosario Peñon, en San Germán.",
    "Estoy buscando información acerca de El Rancho en Palmarejo.",
    "Necesito saber de mi papá vive solo, justo al lado del supermercado Alcoba en Palmarejo un segundo piso casa con techo de zinc",
    "Alguien que me diga respecto a Las Bromelias en Barranquitas?",
    "busco información del barrio higuillar en el area frente a granas lindas",
    "Hay fam. alla y en Canales de Hato Rey.",
    "Area de los Los Guardias, El Rancho Palmarejo Corozal.",
    "Todavia nada de Dorado Calle Norte?",
    "Sector los llanos en Palmarejo no visto ayuda alguna mi hermana tiene MA's de doce personas a su cargo incluyendo Menores.",
]
# test_sent = "Mi familia y vecinos en el rio de Palmarejo, Corozal"

spacy_trees = []
stanford_trees = []
google_trees = []

for test_sent in test_sents:
    print('parsing sentence "%s"'%(test_sent))
    ## spacy
#     print('testing spacy parser')
    spacy_doc = spacy_nlp(test_sent)
    spacy_tree = build_parse(spacy_doc, 'spacy')
    spacy_trees.append(spacy_tree)
#     print(spacy_doc.print_tree())

    ## Stanford
#     print('testing Stanford parser')
    stanford_parse = stanford_parser.parse_one(stanford_parser.tokenize(test_sent))
    stanford_tree = build_parse(stanford_parse, 'stanford')
    stanford_trees.append(stanford_tree)
#     print(next(stanford_parse))
    
    ## Google SyntaxNet
#     print('testing Google parser')
    google_doc = language.types.Document(
        content=test_sent,
        type=language.enums.Document.Type.PLAIN_TEXT
    )
    google_parse = google_client.analyze_syntax(google_doc, retry=False)
    google_tree = build_parse(google_parse, 'google')
    google_trees.append(google_tree)

parsing sentence "Ella vive en Palo Hincado, sector Puente Roto."
parsing sentence "Hola, mi esposo Efrain J Irizarry y su hermano Nelson Irizarry andan buscando a su Papa y su esposa, Mayra Rodriguez dueños de La Guarida del Penon, en Rosario Peñon, en San Germán."
parsing sentence "Estoy buscando información acerca de El Rancho en Palmarejo."
parsing sentence "Necesito saber de mi papá vive solo, justo al lado del supermercado Alcoba en Palmarejo un segundo piso casa con techo de zinc"
parsing sentence "Alguien que me diga respecto a Las Bromelias en Barranquitas?"
parsing sentence "busco información del barrio higuillar en el area frente a granas lindas"
parsing sentence "Hay fam. alla y en Canales de Hato Rey."
parsing sentence "Area de los Los Guardias, El Rancho Palmarejo Corozal."
parsing sentence "Todavia nada de Dorado Calle Norte?"
parsing sentence "Sector los llanos en Palmarejo no visto ayuda alguna mi hermana tiene MA's de doce personas a su cargo incluyendo Menores."


In [74]:
for sent_i, spacy_i, stanford_i, google_i in zip(test_sents, spacy_trees, stanford_trees, google_trees):
    print('test sentence = "%s"'%(sent_i))
    print('spacy')
    print(spacy_i)
    print('stanford')
    print(stanford_i)
    print('google')
    print(google_i)

test sentence = "Ella vive en Palo Hincado, sector Puente Roto."
spacy
[['Ella', 'PRON', 1, 'nsubj'], ['vive', 'VERB', 1, 'ROOT'], ['en', 'ADP', 3, 'case'], ['Palo', 'PROPN', 1, 'obl'], ['Hincado', 'PROPN', 3, 'flat'], [',', 'PUNCT', 6, 'punct'], ['sector', 'NOUN', 3, 'appos'], ['Puente', 'PROPN', 6, 'appos'], ['Roto', 'PROPN', 7, 'flat'], ['.', 'PUNCT', 1, 'punct']]
stanford
[['Ella', 'PRON', 2, 'nsubj'], ['vive', 'VERB', 0, 'ROOT'], ['en', 'ADP', 4, 'case'], ['Palo', 'PROPN', 2, 'nmod'], ['Hincado', 'ADJ', 4, 'amod'], [',', 'PUNCT', 7, 'punct'], ['sector', 'NOUN', 4, 'appos'], ['Puente', 'PROPN', 7, 'appos'], ['Roto', 'PROPN', 8, 'name'], ['.', 'PUNCT', 2, 'punct']]
google
[['Ella', 'PRON', 1, 'nsubj'], ['vive', 'VERB', 1, 'root'], ['en', 'ADP', 1, 'prep'], ['Palo', 'PROPN', 2, 'pobj'], ['Hincado', 'PROPN', 3, 'appos'], [',', 'PUNCT', 3, 'p'], ['sector', 'NOUN', 3, 'conj'], ['Puente', 'PROPN', 3, 'prep'], ['Roto', 'PROPN', 7, 'pobj'], ['.', 'PUNCT', 1, 'p']]
test sentence = "Hola, mi

In [70]:
import networkx as nx
# define method for extracting dependency relations
def extract_dep(tree, dedup_subtrees=False, noun_types = ['NOUN', 'PROPN']):
    # identify PROPN, then identify any attached appositive/PP phrases
    tree = [x + [i] for i,x in enumerate(tree)] # add index for bookkeeping
    np_heads = list(filter(lambda x: x[1] in noun_types, tree))
#     print('got np heads %s'%(str(np_heads)))
    np_descendants = []
    ## TODO: build graph, then extract subtrees?? would be cleaner than this
    tree_graph = nx.DiGraph()
    for i, dep in enumerate(tree):
        tree_graph.add_edge(dep[2], i, pos=dep[1], name='/'.join(map(str, dep)), dep=dep)
    for np_head_i in np_heads:
        idx = np_head_i[4]
        # collect descendents
        desc_i = nx.descendants(tree_graph, idx)
        desc_i = [np_head_i] + [tree[j] for j in desc_i]
        # sort by sentence order
        desc_i = sorted(desc_i, key=lambda x: x[4])
        np_descendants.append(desc_i)
    # get rid of subordinate (duplicate) trees
    if(dedup_subtrees):
        np_descendants_dedup = []
        np_descendants_dedup_str = []
        np_descendants_str = list(map(lambda x: list(map(lambda y: '/'.join(map(str, y)), x)), np_descendants))
        for desc_i in sorted(np_descendants, key=lambda x: len(x), reverse=True):
    #         print('testing desc %s'%(desc_i))
            desc_str = list(map(lambda x: '/'.join(map(str, x)), desc_i))
            prev_match = False
            for np_descendants_dedup_str_i in np_descendants_dedup_str:
                if(len(set(np_descendants_dedup_str_i) - set(desc_str)) < len(np_descendants_dedup_str_i)):
                    prev_match = True
                    break
            if(not prev_match):
    #             print('adding desc %s'%(desc_str))
                np_descendants_dedup_str.append(desc_str)
                np_descendants_dedup.append(desc_i)
        np_descendants = np_descendants_dedup
    return np_descendants
tree_i = spacy_trees[0]
def pretty_print_tree(tree):
    tree_str = ' '.join(map(lambda x: '/'.join(map(str, x)), tree))
    return tree_str
print(pretty_print_tree(tree_i))
descendants = extract_dep(tree_i)
# print(descendants)
for des in descendants:
    print(des)

Ella/PRON/1/nsubj vive/VERB/1/ROOT en/ADP/3/case Palo/PROPN/1/obl Hincado/PROPN/3/flat ,/PUNCT/6/punct sector/NOUN/3/appos Puente/PROPN/6/appos Roto/PROPN/7/flat ./PUNCT/1/punct
[['en', 'ADP', 3, 'case', 2], ['Palo', 'PROPN', 1, 'obl', 3], ['Hincado', 'PROPN', 3, 'flat', 4], [',', 'PUNCT', 6, 'punct', 5], ['sector', 'NOUN', 3, 'appos', 6], ['Puente', 'PROPN', 6, 'appos', 7], ['Roto', 'PROPN', 7, 'flat', 8]]
[['Hincado', 'PROPN', 3, 'flat', 4]]
[[',', 'PUNCT', 6, 'punct', 5], ['sector', 'NOUN', 3, 'appos', 6], ['Puente', 'PROPN', 6, 'appos', 7], ['Roto', 'PROPN', 7, 'flat', 8]]
[['Puente', 'PROPN', 6, 'appos', 7], ['Roto', 'PROPN', 7, 'flat', 8]]
[['Roto', 'PROPN', 7, 'flat', 8]]


Let's run the test sentences through the three parsers and see which parser detects the most of these appositives.

1. `Palo Hincado, sector Puente Roto`
2. `La Guarida del Penon, en Rosario Peñon, en San Germán`
3. `El Rancho en Palmarejo`
4. `supermercado Alcoba en Palmarejo`
5. `Las Bromelias en Barranquitas`
6. `barrio higuillar en el area frente a granas lindas`
7. `Canales de Hato Rey`
8. `Los Guardias, El Rancho Palmarejo Corozal`
9. `Dorado Calle Norte`
10. `Sector los llanos en Palmarejo`

In [71]:
test_appos = [
    'Palo Hincado , sector Puente Roto',
    'La Guarida del Penon , en Rosario Peñon , en San Germán',
    'El Rancho en Palmarejo',
    'supermercado Alcoba en Palmarejo',
    'Las Bromelias en Barranquitas',
    'barrio higuillar en el area frente a granas lindas',
    'Canales de Hato Rey',
    'Los Guardias , El Rancho Palmarejo Corozal',
    'Dorado Calle Norte',
    'Sector los llanos en Palmarejo'
]

def get_all_tree_str(x):
    x_str = [' '.join([z[0] for z in y]) for y in x]
    return x_str

## exact matching
spacy_ctr = 0
stanford_ctr = 0
google_ctr = 0
for i, (test_sent_i, test_appos_i, spacy_tree_i, stanford_tree_i, google_tree_i) in enumerate(zip(test_sents, test_appos, spacy_trees, stanford_trees, google_trees)):
    print('%d. %s'%(i+1, test_sent_i))
    spacy_descendants_i = extract_dep(spacy_tree_i)
    spacy_descendants_str_i = get_all_tree_str(spacy_descendants_i)
    spacy_ctr += int(test_appos_i in spacy_descendants_str_i)
    print('spacy dep\n%s'%('\n'.join(map(pretty_print_tree, spacy_descendants_i))))
    stanford_descendants_i = extract_dep(stanford_tree_i)
    stanford_descendants_str_i = get_all_tree_str(stanford_descendants_i)
    stanford_ctr += int(test_appos_i in stanford_descendants_str_i)
    print('stanford dep\n%s'%('\n'.join(map(pretty_print_tree, stanford_descendants_i))))
    google_descendants_i = extract_dep(google_tree_i)
    google_descendants_str_i = get_all_tree_str(google_descendants_i)
    google_ctr += int(test_appos_i in google_descendants_str_i)
    print('google dep\n%s'%('\n'.join(map(pretty_print_tree, google_descendants_i))))
    print('~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~')
#     print("spacy dep\n%s"%(spacy_descendants_i))
#     print("stanford dep\n%s"%(stanford_descendants_i))
#     print("google dep\n%s"%(google_descendants_i))
print('spacy %d/%d'%(spacy_ctr, len(test_sents)))
print('stanford %d/%d'%(stanford_ctr, len(test_sents)))
print('google %d/%d'%(google_ctr, len(test_sents)))

1. Ella vive en Palo Hincado, sector Puente Roto.
spacy dep
en/ADP/3/case/2 Palo/PROPN/1/obl/3 Hincado/PROPN/3/flat/4 ,/PUNCT/6/punct/5 sector/NOUN/3/appos/6 Puente/PROPN/6/appos/7 Roto/PROPN/7/flat/8
Hincado/PROPN/3/flat/4
,/PUNCT/6/punct/5 sector/NOUN/3/appos/6 Puente/PROPN/6/appos/7 Roto/PROPN/7/flat/8
Puente/PROPN/6/appos/7 Roto/PROPN/7/flat/8
Roto/PROPN/7/flat/8
stanford dep
Palo/PROPN/2/nmod/3
sector/NOUN/4/appos/6
,/PUNCT/7/punct/5 Puente/PROPN/7/appos/7
Roto/PROPN/8/name/8
google dep
Palo/PROPN/2/pobj/3 Hincado/PROPN/3/appos/4 ,/PUNCT/3/p/5 sector/NOUN/3/conj/6 Puente/PROPN/3/prep/7 Roto/PROPN/7/pobj/8
Hincado/PROPN/3/appos/4
sector/NOUN/3/conj/6
Puente/PROPN/3/prep/7 Roto/PROPN/7/pobj/8
Roto/PROPN/7/pobj/8
~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~
2. Hola, mi esposo Efrain J Irizarry y su hermano Nelson Irizarry andan buscando a su Papa y su esposa, Mayra Rodriguez dueños de La Guarida del Penon, en Rosario Peñon, en San Germán.
spacy dep
Hola/PROPN/13/nsubj/

Bottom line: Google has the most consistent appositive extraction, as long as we don't deduplicate the dependency trees.

In [ ]:
%%capture parse_output
from google.cloud import language
import time
import nltk
google_cred_file = '../../data/Google_cloud/TestProject_creds.json'
google_client = language.LanguageServiceClient.from_service_account_json(google_cred_file)
sent_tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')
entity_data_tagged_sents = entity_data_tagged.loc[:, 'status_message_clean']
entity_data_ids = entity_data_tagged.loc[:, 'status_id']

google_trees_complete = []
out_file_name = '../../data/facebook-maria/combined_group_data_es_tagged_parsed.txt'
## TODO: check for existing statuses, make sure not to parse
SLEEP_TIME = 30.
SLEEP_CTR = 1000
tree_ctr = 0
err = None
with open(out_file_name, 'w') as out_file:
    for idx, txt in zip(entity_data_ids, entity_data_tagged_sents):
        sents = sent_tokenizer.tokenize(txt)
        for sent in sents:
            google_doc = language.types.Document(content=sent, type=language.enums.Document.Type.PLAIN_TEXT, language='es')
            parse_success = False
            while(not parse_success):
                try:
                    google_parse = google_client.analyze_syntax(google_doc, retry=False)
                    parse_success = True
                except Exception as e:
                    # avoid rate limiting
                    err = e
                    print('exception %s'%(e))
                    print('sleeping %.3f'%(SLEEP_TIME))
                    time.sleep(SLEEP_TIME)
                    SLEEP_CTR -= 1
                    if(SLEEP_CTR <= 0):
                        parse_success = True
                        break
            google_tree = build_parse(google_parse, 'google')
            google_tree_str = ' '.join(['/'.join(map(str, x)) for x in google_tree])
            out_file.write('%s\t%s\n'%(idx, google_tree_str))
            if(tree_ctr % 100 == 0):
                print('processed %d sents'%(tree_ctr))
            google_trees_complete.append(google_tree)
            tree_ctr += 1
        if(SLEEP_CTR <= 0):
            break

In [76]:
parsed_data = pd.read_csv('../../data/facebook-maria/combined_group_data_es_tagged_parsed.txt', sep='\t', index_col=False, names=['status_id', 'status_parsed'])
print('%d sentences parsed'%(parsed_data.shape[0]))
display(parsed_data.head())

48439 sentences parsed


,status_id,status_parsed
0,351272391991842_362333787552369,Estoy/VERB/1/aux preocupada/VERB/1/root por/AD...
1,351272391991842_361453717640376,En/ADP/0/root apoyo/NOUN/0/pobj al/ADP/1/prep ...
2,351272391991842_358929311226150,Pueblo/NOUN/6/nsubj de/ADP/0/prep Puerto/PROPN...
3,351272391991842_358929311226150,No/ADV/2/neg se/PRON/2/prt metan/VERB/2/root e...
4,351272391991842_357797888005959,"Guayama/PROPN/0/root ,/PUNCT/0/p Dorado/ADJ/0/..."


Let's extract some appositions and descriptors using the regular expressions listed by Staliunaite et al.:

- NP -> S-BAR (`relcl`)
- NP -> VP (`acl`)
- NP -> NP (`appos`)
- NP -> ADJP (`amod`)
- NP -> PP (`prep`)

I think that only NP->NP and NP->PP will produce useful appositives, the rest will be less reliable. Let's also use all NOUN words to start and whittle them down later.

In [227]:
# filter for NOUNS/PROPN whose
# immediate child is relcl/acl/appos/amod/prep
DESCRIPTOR_CHILD_CATEGORIES = ['relcl', 'acl', 'appos', 'amod', 'prep']

def filter_children(children):
    """
    Filter children according to their category: we 
    want NPs with descriptor categories (appos, etc.).
    
    :param children: list of dependency tuples
    :returns children_filtered: list of dependency tuples
    """
    # one subtree at a time!
    # determine parent noun
    children_sorted = sorted(children, key=lambda x: x[4])
#     print('sorted children')
#     print(children_sorted)
    parent_idx = min(children, key=lambda y: y[2])[4]
#     print(parent_idx)
    # find most immediate right child and see if it matches allowed category
#     print(list(filter(lambda y: y[2] == parent_idx and y[4] > parent_idx, children_sorted)))
    matching_children = list(filter(lambda y: y[2] == parent_idx and y[4] > parent_idx and y[3] in DESCRIPTOR_CHILD_CATEGORIES, children_sorted))
    children_filtered = []
    if(len(matching_children) > 0):
        # tmp: only using right-most children is too restrictive!
#         print(matching_children)
#         right_child = matching_children[0]
#         right_child_idx = right_child[4]
#         children_filtered = list(filter(lambda y: y[4] == parent_idx or y[4] >= right_child_idx, children_sorted))
        children_filtered = list(children)
#         if(len(matching_children) > 1):
#             print('%d children to parent, abort'%(len(matching_children)))
#             print(children_filtered)
        
    return children_filtered

def collect_filter_children(children_list):
    children_list_filtered = list(filter(lambda x: len(x) > 0, map(filter_children, children_list)))
    return children_list_filtered

In [228]:
## does the filtering recover these sentences??
test_children_sents = [
    "El/PROPN/6/det/5 Rancho/PROPN/4/pobj/6 en/ADP/6/prep/7 Palmarejo/PROPN/7/pobj/8",
    "Las/PROPN/7/det/6 Bromelias/PROPN/4/pobj/7 en/ADP/7/prep/8 Barranquitas/PROPN/8/pobj/9",
    "Sector/NOUN/6/nsubj/0 los/DET/2/det/1 llanos/NOUN/0/dobj/2 en/ADP/0/prep/3 Palmarejo/PROPN/3/pobj/4", # fails on this one because multiple parents!! 
]
for test_children_sent in test_children_sents:
    test_children = [x.split('/') for x in test_children_sent.split(' ')]
    test_children = [[x[0], x[1], int(x[2]), x[3], int(x[4])] for x in test_children]
    print(test_children)
    test_filtered_children = filter_children(test_children)
    print(test_filtered_children)

[['El', 'PROPN', 6, 'det', 5], ['Rancho', 'PROPN', 4, 'pobj', 6], ['en', 'ADP', 6, 'prep', 7], ['Palmarejo', 'PROPN', 7, 'pobj', 8]]
[['El', 'PROPN', 6, 'det', 5], ['Rancho', 'PROPN', 4, 'pobj', 6], ['en', 'ADP', 6, 'prep', 7], ['Palmarejo', 'PROPN', 7, 'pobj', 8]]
[['Las', 'PROPN', 7, 'det', 6], ['Bromelias', 'PROPN', 4, 'pobj', 7], ['en', 'ADP', 7, 'prep', 8], ['Barranquitas', 'PROPN', 8, 'pobj', 9]]
[['Las', 'PROPN', 7, 'det', 6], ['Bromelias', 'PROPN', 4, 'pobj', 7], ['en', 'ADP', 7, 'prep', 8], ['Barranquitas', 'PROPN', 8, 'pobj', 9]]
[['Sector', 'NOUN', 6, 'nsubj', 0], ['los', 'DET', 2, 'det', 1], ['llanos', 'NOUN', 0, 'dobj', 2], ['en', 'ADP', 0, 'prep', 3], ['Palmarejo', 'PROPN', 3, 'pobj', 4]]
[]


In [229]:
# ## collect children
parse_children = []
for x in parsed_data.loc[:, 'status_parsed'].values:
    x_parse = [y.split('/') for y in x.split(' ')]
    x_parse = list(filter(lambda x: len(x) == 4, x_parse))
    # fix numbers 
    x_parse = [[y[0], y[1], int(y[2]), y[3]] for y in x_parse]
    x_children = extract_dep(x_parse, noun_types=['PROPN'])
    parse_children.append(x_children)

parse_children_filtered = []
for parse_children_i in parse_children:
#     print(x)
#     print('filtering children')
    parse_children_filtered_i = collect_filter_children(parse_children_i)
#     print('collected children')
#     print(x_children)
    parse_children_filtered.append(parse_children_filtered_i)

In [230]:
parsed_data.loc[:, 'parse_children_filtered'] = parse_children_filtered
# convert to strings
parsed_data.loc[:, 'parse_children_filtered_str'] = parsed_data.loc[:, 'parse_children_filtered'].apply(lambda x: [pretty_print_tree(y) for y in x]).values

In [231]:
display(parsed_data.head())

,status_id,status_parsed,parse_children_filtered,parse_children_filtered_str
0,351272391991842_362333787552369,Estoy/VERB/1/aux preocupada/VERB/1/root por/AD...,[],[]
1,351272391991842_361453717640376,En/ADP/0/root apoyo/NOUN/0/pobj al/ADP/1/prep ...,"[[[Dorado, PROPN, 9, pobj, 10], [Dorado, PROPN...",[Dorado/PROPN/9/pobj/10 Dorado/PROPN/10/amod/11]
2,351272391991842_358929311226150,Pueblo/NOUN/6/nsubj de/ADP/0/prep Puerto/PROPN...,[],[]
3,351272391991842_358929311226150,No/ADV/2/neg se/PRON/2/prt metan/VERB/2/root e...,"[[[los, DET, 13, det, 12], [Estados, PROPN, 11...",[los/DET/13/det/12 Estados/PROPN/11/pobj/13 Un...
4,351272391991842_357797888005959,"Guayama/PROPN/0/root ,/PUNCT/0/p Dorado/ADJ/0/...",[],[]


How do the extracted appositive phrases look?

In [232]:
parsed_data_filtered = parsed_data[parsed_data.loc[:, 'parse_children_filtered'].apply(len) > 0]
print("%d non-null appositives"%(len(parsed_data_filtered)))
print(parsed_data_filtered.loc[:, 'parse_children_filtered_str'].head(20).apply(lambda x: '\n'.join(x)).values)

10703 non-null appositives
['Dorado/PROPN/9/pobj/10 Dorado/PROPN/10/amod/11'
 'los/DET/13/det/12 Estados/PROPN/11/pobj/13 Unidos/PROPN/13/amod/14 de/ADP/13/prep/15 America/PROPN/15/pobj/16\nlos/DET/36/det/35 Estados/PROPN/34/pobj/36 Unidos/PROPN/36/amod/37'
 'AVISO/PROPN/1/parataxis/3 IMPORTANTEA/PROPN/3/appos/4 todas/DET/7/predet/5 las/DET/7/det/6 personas/NOUN/3/appos/7 de/ADP/7/prep/8 Puerto/PROPN/8/pobj/9 Rico/PROPN/9/nn/10 que/PRON/12/nsubj/11 quieran/VERB/7/rcmod/12 venir/VERB/12/xcomp/13 a/ADP/13/prep/14 la/DET/16/det/15 Florida/PROPN/14/pobj/16 tras/ADP/13/prep/17 el/DET/19/det/18 paso/NOUN/17/pobj/19 del/ADP/19/prep/20 huracán/NOUN/20/pobj/21 Maria/PROPN/21/appos/22 ,/PUNCT/12/p/23 abrieron/VERB/12/parataxis/24 una/DET/26/det/25 oficina/NOUN/24/dobj/26 para/ADP/24/prep/27 brindar/VERB/27/pcomp/28 les/PRON/30/iobj/29 ayuda/VERB/24/parataxis/30 mientras/ADV/32/advmod/31 están/VERB/30/advcl/32 acá/ADV/32/advmod/33'
 'Centro/PROPN/3/pobj/4 de/ADP/4/prep/5 Ayuda/PROPN/5/pobj/6 y/CO

To sanity check: what are the most common head nouns?

In [233]:
from collections import Counter
head_np_counts = Counter()
head_bigram_counts = Counter()
head_np_joint_counts = Counter()
for x in parsed_data_filtered.loc[:, 'parse_children_filtered'].values:
    x_heads = [y[0] for y in x]
    for x_head in x_heads:
        head_np_counts[x_head[0]] += 1
    x_bigrams = [[y[0][0], y[1][0]] for y in x]
    for x_bigram in x_bigrams:
        head_bigram_counts['_'.join(x_bigram)] += 1
    for y in x:
        y_joint_head = []
        y_joint_head_queue = [] # keep running queue for prepositions
        for dep_token in y:
            if(dep_token[1] == 'PROPN'):
                y_joint_head += y_joint_head_queue
                y_joint_head.append(dep_token[0])
                y_joint_head_queue = []
            elif(dep_token[1] == 'ADP'):
                y_joint_head_queue.append(dep_token[0])
            else:
                break
        y_joint_head = '_'.join(y_joint_head)
        head_np_joint_counts[y_joint_head] += 1
head_np_counts = pd.Series(head_np_counts).sort_values(inplace=False, ascending=False)
head_bigram_counts = pd.Series(head_bigram_counts).sort_values(inplace=False, ascending=False)
head_np_joint_counts = pd.Series(head_np_joint_counts).sort_values(inplace=False, ascending=False)
print('head NNP counts')
display(head_np_counts.head(50))
print('head bigram counts')
display(head_bigram_counts.head(50))
print('head joint NNP counts')
display(head_np_joint_counts.head(50))
display(head_np_joint_counts.tail(50))

head NNP counts


la              1212
Barrio           640
el               635
Vega             588
,                385
mi               258
Calle            255
La               226
Villa            220
El               197
los              171
Sector           159
Urbanización     151
(                141
#                137
the              120
Jardines         120
Los              113
Las               93
my                91
Mi                86
Familia           84
Dios              78
Pueblo            73
Luis              71
las               70
Maria             69
Don               68
Carmen            67
San               62
you               61
Estados           60
Jose              60
Aguas             60
Brisas            60
"                 59
Camino            59
from              53
Cerro             53
Carretera         52
Centro            51
Huracán           51
Puerto            49
Guayama           49
Toa               48
Río               45
Palo              44
URL          

head bigram counts


Vega_Alta          307
la_Urbanización    243
Vega_Baja          223
el_Barrio          205
la_Calle           123
Jardines_de         78
El_Barrio           71
la_Familia          59
Estados_Unidos      55
Aguas_Buenas        55
la_Iglesia          54
los_Estados         50
Cerro_Gordo         44
la_Barriada         43
la_Carretera        37
Camino_Nuevo        37
Villa_Madrid        36
PUERTO_RICO         36
Villa_Rosa          36
Pueblo_Nuevo        35
Palo_Hincado        34
las_Parcelas        33
Alturas_de          31
Vega_alta           31
Brisas_del          29
mi_Tia              29
Centro_de           29
Costa_Azul          29
el_Sector           28
Toa_Baja            26
,_Vega              26
Huracán_María       26
Jardines_del        25
el_Centro           25
Valles_de           25
mi_Tio              24
Trujillo_Alto       23
Puerto_Rico         23
,_Sector            23
la_plaza            23
Brisas_de           22
Vega_baja           22
any_information     22
Pueblo_de  

head joint NNP counts


                        3654
Vega_Alta                294
Vega_Baja                208
Dios                      66
#                         58
Estados_Unidos            55
Vega                      52
Aguas_Buenas              52
Cerro_Gordo               38
Camino_Nuevo              34
Villa_Madrid              29
PUERTO_RICO               28
Palo_Hincado              28
Pueblo_Nuevo              28
Guayama                   27
Costa_Azul                26
Toa_Baja                  23
Puerto_Rico               20
calle                     20
Barranquitas              19
from                      19
URL                       19
Toa_Alta                  19
Ciudad_Universitaria      17
Gracias                   17
Ponce                     17
Coamo                     17
PR                        17
my                        17
Favor                     16
Brisas_del_Mar            16
Carmen                    16
Gloria_a_Dios             16
Carretera                 15
Jardines      

Barrio_Corazon_Calle_Flamboyan                                    1
AEE                                                               1
Delia_Maria_Rivera                                                1
Guayama_de_Urbanizaciónbrisas                                     1
Enchautegui                                                       1
Debra_Ledee_en_Enchautegui                                        1
Luz_Delia                                                         1
Recidencial_Jardines_De_Guamani                                   1
Comunicarnos                                                      1
Mi_Tío_Nolyn_Guadalupe                                            1
Mi_Mama_Lucy_Guadalupe                                            1
brisas_Del_Mar                                                    1
Urbanizaciónpaseo_brisas_Del_Mar                                  1
Jose_Miranda_Rodriguez                                            1
anyone_knows_anything_at_all_about_those        

How many of the head NPs can we find in the gazetteer and are relatively unambiguous?

In [234]:
from unidecode import unidecode
geonames_data = pd.read_csv('/hg190/corpora/GeoNames/allCountriesSimplified.tsv', sep='\t', index_col=False)
geonames_data.loc[:, 'name_norm'] = geonames_data.loc[:, 'name'].apply(lambda x: unidecode(x.lower()))
# display(geonames_data.head())
# limit to PR data
geonames_data_PR = geonames_data[geonames_data.loc[:, 'country']=='PR']
display(geonames_data_PR.head(50))

,geonames_ID,name,alternate_names,latitude,longitude,feature_class,feature_code,country,population,alternate_name_count,name_norm
7739099,4049880,Teresita,NaN,18.40967,-66.17933,P,PPL,PR,0,1,teresita
7739100,4049900,Rio Cristal,NaN,18.35328,-66.00322,P,PPL,PR,0,1,rio cristal
7739101,4049922,Quebrada Rincón,Quebrada Rincon,18.28551,-65.69461,T,VAL,PR,0,1,quebrada rincon
7739102,4050022,Lago Gely,NaN,18.04719,-66.57184,H,LK,PR,0,1,lago gely
7739103,4050076,Glenview,NaN,18.04608,-66.59378,P,PPL,PR,0,1,glenview
7739104,4050096,Caño Boquernón,"Cano Boquernon,Cano Boqueron",18.00774,-67.17379,H,LK,PR,0,2,cano boquernon
7739105,4050120,Canal del Medio,NaN,18.43550,-65.92517,H,CNL,PR,0,1,canal del medio
7739106,4050312,Pamorama,NaN,18.35606,-66.18406,P,PPL,PR,0,1,pamorama
7739107,4050320,Nola,NaN,18.47050,-66.72018,P,PPL,PR,0,1,nola
7739108,4050322,Coabey,NaN,18.20746,-66.56073,P,PPL,PR,0,1,coabey


In [235]:
head_np_joint_loc_counts = pd.Series({x: geonames_data_PR[geonames_data_PR.loc[:, 'name_norm'] == unidecode(x.replace('_', ' ').lower())].shape[0] for x in head_np_joint_counts.index})
head_np_joint_loc_counts = head_np_joint_loc_counts[head_np_joint_loc_counts > 0]
head_np_joint_loc_counts.sort_values(inplace=True, ascending=False)
head_np_joint_loc_counts_unambiguous = head_np_joint_loc_counts[head_np_joint_loc_counts == 1]
print(head_np_joint_loc_counts)
print('%d/%d unambiguous head NNPs'%(len(head_np_joint_loc_counts_unambiguous), len(head_np_joint_loc_counts)))

Quebrada_Grande                25
Rio_Canas                      10
Río_Canas                      10
San_Antonio                    10
san_José                        8
San_Jose                        8
Cerro_Gordo                     7
CERRO_GORDO                     7
Rio_Hondo                       6
Las_Mercedes                    6
Río_Hondo                       6
Centro_de_Bellas_Artes          5
Las_Palmas                      5
Santa_Clara                     5
Las_PALMAS                      5
Palmarejo                       5
Punta_Brava                     5
palmarejo                       5
Sabana                          5
Rio_Grande                      4
Rio_Blanco                      4
Santa_Rosa                      4
Cuatro_Calles                   4
Las_Flores                      4
Valencia                        4
Río_Piedras                     4
Rio_Piedras                     4
Villa_del_Carmen                4
Pueblo_Nuevo                    4
La_Aldea      

OK! This gives us enough to work with for a first pass.

For the unambiguous locations, are the attached phrases being used for extra information or what?

In [236]:
# examine descriptor phrases for head NPs
from collections import defaultdict
head_np_descriptions = defaultdict(list)
parse_children_str_combined = []
for parse_children in parsed_data_filtered.loc[:, 'parse_children_filtered'].values:
    for parse_child in parse_children:
        joint_head = []
        joint_head_queue = [] # keep running queue for prepositions
        dep_ctr = 0
        for dep_token in parse_child:
            if(dep_token[1] == 'PROPN'):
                joint_head += joint_head_queue
                joint_head.append(dep_token[0])
                joint_head_queue = []
                dep_ctr = len(joint_head)
            elif(dep_token[1] == 'ADP'):
                joint_head_queue.append(dep_token[0])
            else:
                break
        joint_head = '_'.join(joint_head)
        head_description = ' '.join([x[0] for x in parse_child[dep_ctr:]])
        if(len(head_description) > 0):
            head_np_descriptions[joint_head].append(head_description)
        parse_children_str_combined.append(' '.join([x[0] for x in parse_child]))
        # debugging Vega Alta
#         if(joint_head == 'Vega_Alta'):
#             print('test parse child')
#             print(parse_child)
#             print('dep ctr %d'%(dep_ctr))
#             print('head description: %s'%(head_description))
#         print('head description %s'%(head_description))
head_np_descriptions = dict(head_np_descriptions)

In [237]:
print(len(head_np_descriptions))
# sort by number of descriptions
head_np_descriptions_items = sorted(head_np_descriptions.items(), key=lambda x: len(set(x[1])), reverse=True)
top_k = 150
for i in range(top_k):
    head_np_i = head_np_descriptions_items[i][0]
    print('head NP %s has %d children'%(head_np_i, len(head_np_descriptions_items[i][1])))
    print('\n'.join(['\t%s %s'%(head_np_i, x) for x in head_np_descriptions_items[i][1]]))

5283
head NP  has 3654 children
	 los Estados Unidos de America
	 los Estados Unidos
	 480 ext Calle sur Dorado PR
	 mi Dios , ayuda a toda mi familia , amistades y a toda mi gente en Puerto Rico a encontrar las fuerzas , la paz y la paciencia para poder lidear con esta pesadilla diaria .... 😔 Señor a ti te imploro con todo mi corazón y toda mi alma que levantes a mi isla querida de Puerto Rico ! !
	 la Urbanización Jardin Dorado ? Nadie de mi familia
	 ? Nadie de mi familia
	 mi Querido Viejo
	 la plaza del Pueblo de Dorado
	 163 Guayacán , hacienda mi querido viejo , dorado puerto Rico
	 la Policia municipal de Dorado
	 la Urbanización Monterrey , Quintas de Monte Rio y parte de Ponce de Leon en Mayaguez ,
	 , Quintas de Monte Rio y parte de Ponce de Leon en Mayaguez ,
	 la Urbanización Ana Maria
	 un Stop and Go para familias del pueblo
	 * Manejo de Emergencias 787-892-5625 787-892-5685 * Policía SG 787-892-3505
	 la Calle Luna
	 El Real Gracias a Dios
	 , Netti Gutierrez la hija ,

This is odd! A lot of these head NPs (`Vega_Alta`, `Aguas_Buenas`, `Guayama`, `Palmarejo`) are actually descriptors for their children (e.g. `Vega_Alta sector Santana`, `Vega_Alta y Vega Baja`, `Guayama barrio Olimpo`, `Palmarejo en las parcelas`) except in cases of "standard" address order (`Vega_Alta, Puerto Rico`).

What about those NPs that we used earlier for comparing the parsers?

In [238]:
from functools import reduce
test_descriptors = [
    'Palo Hincado , sector Puente Roto',
    'La Guarida del Penon , en Rosario Peñon , en San Germán',
    'El Rancho en Palmarejo',
    'supermercado Alcoba en Palmarejo',
    'Las Bromelias en Barranquitas',
    'barrio higuillar en el area frente a granas lindas',
    'Canales de Hato Rey',
    'Los Guardias , El Rancho Palmarejo Corozal',
    'Dorado Calle Norte',
    'Sector los llanos en Palmarejo'
]
head_np_descriptors_str = [['%s %s'%(x[0], y) for y in x[1]] for x in head_np_descriptions_items]
head_np_descriptors_str = list(reduce(lambda x,y: x+y,  head_np_descriptors_str))
print('%d descriptor phrases'%(len(head_np_descriptors_str)))
for test_descriptor in test_descriptors:
    if(test_descriptor in head_np_descriptors_str):
        print('head NP found: %s'%(test_descriptor))
    if(test_descriptor in parse_children_str_combined):
        print('descriptor str found: %s'%(test_descriptor))

11146 descriptor phrases
descriptor str found: Palo Hincado , sector Puente Roto
descriptor str found: El Rancho en Palmarejo
descriptor str found: Las Bromelias en Barranquitas
descriptor str found: Canales de Hato Rey
descriptor str found: Dorado Calle Norte


For all unambiguous locations, let's figure out what groups they tend to occur in and how long their descriptors are.

In [127]:
parsed_data_filtered.loc[:, 'group_id'] = parsed_data_filtered.loc[:, 'status_id'].apply(lambda x: )

,status_id,status_parsed,parse_children_filtered,parse_children_filtered_str
1,351272391991842_361453717640376,En/ADP/0/root apoyo/NOUN/0/pobj al/ADP/1/prep ...,"[[[Dorado, PROPN, 9, pobj, 10], [Dorado, PROPN...",[Dorado/PROPN/9/pobj/10 Dorado/PROPN/10/amod/11]
6,351272391991842_356016884850726,🇵/PUNCT/1/det 🇷/X/1/root ..../PUNCT/1/p AVISO/...,"[[[AVISO, PROPN, 1, parataxis, 3], [IMPORTANTE...",[AVISO/PROPN/1/parataxis/3 IMPORTANTEA/PROPN/3...
9,351272391991842_356016884850726,El/DET/1/det número/NOUN/9/attr telefónico/ADJ...,"[[[Centro, PROPN, 3, pobj, 4], [de, ADP, 4, pr...",[Centro/PROPN/3/pobj/4 de/ADP/4/prep/5 Ayuda/P...
10,351272391991842_356016884850726,Pueden/VERB/1/aux llamar/VERB/1/root para/ADP/...,"[[[Paste, PROPN, 5, conj, 7], [Share, PROPN, 7...",[Paste/PROPN/5/conj/7 Share/PROPN/7/appos/9 )/...
13,351272391991842_355121381606943,Es/VERB/0/root de/ADP/0/prep Utuado/PROPN/1/po...,"[[[Toa, PROPN, 5, pobj, 6], [Baja, PROPN, 6, a...",[Toa/PROPN/5/pobj/6 Baja/PROPN/6/amod/7]


In [125]:
combined_group_data = pd.read_csv('../../data/facebook-maria/combined_group_data_es_tagged.tsv', sep='\t', index_col=False)
display(combined_group_data.head())

,group_id,status_author_id,status_message,status_lang,status_id,status_published,status_message_clean,status_message_tags,status_message_tags_ne,status_message_ne_tagged_stemmed
0,351272391991842,10159623474155515,Estoy preocupada por mi tia vicky vazquez que ...,es,351272391991842_362333787552369,2017-10-19 18:25:23,Estoy preocupada por mi tia vicky vazquez que ...,"[('Estoy', 'O'), ('preocupada', 'O'), ('por', ...","[('asma', 'CAUSE_OF_DEATH'), ('Dorado', 'ORGAN...",estoy preocup por mi tia vicky vazquez que des...
1,351272391991842,10155877836122783,Para alguien que lo necesite... Dale Share..A ...,es,351272391991842_361695457616202,2017-10-18 06:47:34,Para alguien que lo necesite... Dale Share..A ...,"[('Para', 'O'), ('alguien', 'O'), ('que', 'O')...","[('Share', 'MISC'), ('general', 'TITLE'), ('Hi...",par algui que lo necesit... da le share.<ne.mi...
2,351272391991842,1716175958400853,En apoyo al alcalde Carlos Lopez y al municipi...,es,351272391991842_361453717640376,2017-10-17 17:38:08,En apoyo al alcalde Carlos Lopez y al municipi...,"[('En', 'O'), ('apoyo', 'O'), ('a', 'O'), ('el...","[('alcalde', 'TITLE'), ('Carlos Lopez', 'PERSO...",en apoy a el alcalde.<ne.title> carlos_lopez.<...
3,351272391991842,141039043205101,Pueblo de Puerto Rico no se monten en las gran...,es,351272391991842_358929311226150,2017-10-11 14:14:22,Pueblo de Puerto Rico no se monten en las gran...,"[('Pueblo', 'O'), ('de', 'O'), ('Puerto', 'COU...","[('Puerto Rico', 'COUNTRY'), ('America', 'MISC...",puebl de puerto_rico.<ne.country> no se mont e...
4,351272391991842,141039043205101,PUEBLO DE PUERTO RICO no esperan ninguna ayuda...,es,351272391991842_358942134558201,2017-10-11 14:37:12,PUEBLO DE PUERTO RICO no esperan ninguna ayuda...,"[('PUEBLO', 'O'), ('DE', 'O'), ('PUERTO', 'O')...","[('Estados Unidos', 'COUNTRY'), ('Puerto Rico'...",puebl de puert ric no esper ningun ayud federa...


When we figure out which parser is best:

- parse all sentences
- extract appositives
- determine rate of appositive use for disambiguation vs. clarification
- restrict head NPs to unambiguous entities
- determine mean descriptor length on head NPs